# Pipeline for High-z Radio Galaxies 12: Create lists (files) with radio AGN candidates

## Introduction

In this file, three models will be applied consecutively in order to predict  
the detection of Radio Galaxies (radio AGN) and their redshift.  

In principle, this pipeline should be applied to data in Stripe 82. But  
it can be used with any other suitable dataset.

In [1]:
%matplotlib inline
# Static plots
#%matplotlib ipympl
# Interactive plots
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib import ticker
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patheffects as mpe
import matplotlib.patches as mpatches
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from astropy.visualization import LogStretch, PowerStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay
import sklearn.pipeline
import colorcet as cc
from pycaret import classification as pyc
from pycaret import regression as pyr
from joblib import dump, load
import pandas as pd
import mpl_scatter_density
import global_variables as gv
import global_functions as gf

In [2]:
mpl.rcdefaults()

---

## Reading data

Flags.

In [3]:
save_plot_flag      = False
save_preds_flag     = False
load_models_flag    = True
predict_only_hi_z   = False
use_zeroth_model    = False
use_second_z_model  = False  # z >= 3.6 (with SMOGN), or, if needed, z >= 4.0

In [4]:
used_area           = 'HETDEX'  # can be 'S82', 'HETDEX', 'COSMOS'

In [5]:
file_name_dict      = {'S82': gv.file_S82, 'HETDEX': gv.file_HETDEX, 'COSMOS': gv.file_COSMOS}
file_name           = file_name_dict[used_area]

In [6]:
feats_2_disc_S82    = ['objID', 'RA_MILLI', 'DEC_MILLI', 'X_ray_detect', 'W1mag', 'W2mag', 'FUVmag', 'NUVmag',\
                       'num_imputed', 'radio_detect', 'TGSS_detect', 'VLASS_detect']
feats_2_disc_HETDEX = ['objID', 'RA_MILLI', 'DEC_MILLI', 'X_ray_detect', 'W1mag', 'W2mag', 'FUVmag', 'NUVmag',\
                       'num_imputed', 'radio_detect', 'TGSS_detect', 'VLASS_detect', 'LoLSS_detect']
feats_2_disc_COSMOS = ['objID', 'RA_MILLI', 'DEC_MILLI', 'X_ray_detect', 'W1mag', 'W2mag', 'FUVmag', 'NUVmag',\
                       'num_imputed', 'radio_detect', 'TGSS_detect', 'VLASS_detect']

feats_2_disc        = {'S82': feats_2_disc_S82, 'HETDEX': feats_2_disc_HETDEX, 'COSMOS': feats_2_disc_COSMOS}
features_2_discard  = feats_2_disc[used_area]

In [7]:
full_catalog_df     = pd.read_hdf(gv.cat_path + file_name, key='df').drop(columns=features_2_discard)

In [8]:
if used_area == 'S82':
    full_catalog_df.loc[:, 'LOFAR_detect'] = full_catalog_df.loc[:, 'VLAS82_detect'].copy()
    full_catalog_df = full_catalog_df.drop(columns=['VLAS82_detect'])
if used_area == 'COSMOS':
    full_catalog_df.loc[:, 'LOFAR_detect'] = full_catalog_df.loc[:, 'COSMOSVLA3_detect'].copy()
    full_catalog_df = full_catalog_df.drop(columns=['COSMOSVLA3_detect'])

Create features with class and combined redshift.

In [9]:
full_catalog_df['class']            = full_catalog_df.loc[:, 'is_AGN'].copy()
filter_non_confirmed                = np.array(full_catalog_df.loc[:, 'is_AGN'] == 1) | np.array(full_catalog_df.loc[:, 'is_gal'] == 1)
full_catalog_df.loc[~filter_non_confirmed, 'class'] = np.nan
idx_non_Z                           = full_catalog_df.loc[:, 'Z'].where(full_catalog_df.loc[:, 'Z'] > 0).isna()
full_catalog_df.loc[idx_non_Z, 'Z'] = full_catalog_df.loc[:, 'Z'].mask(idx_non_Z, full_catalog_df.loc[idx_non_Z, 'zsp'])

Create column for detection as Radio AGN

In [10]:
full_catalog_df['radio_AGN']      = (np.array(full_catalog_df.loc[:, 'is_AGN'] == 1) & np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1)).astype(int)

Discard minor features.

In [11]:
full_catalog_df                     = full_catalog_df.drop(columns=['is_AGN', 'is_SDSS_QSO', 'is_SDSS_gal', 'is_gal', 'zsp'])

#### As we want to predict, only use sources that have not previous spectroscopic classification

In [12]:
print(f'The used data set (in {used_area}) has {len(full_catalog_df):,} sources.')
print(f'And {np.sum(filter_non_confirmed):,} have previous spectroscopic classification.')

The used data set (in HETDEX) has 6,729,647 sources.
And 83,409 have previous spectroscopic classification.


In [13]:
full_catalog_df                     = full_catalog_df.loc[~filter_non_confirmed]

In [14]:
print(f'This pipeline will predict properties for {len(full_catalog_df):,} sources in {used_area}.')

This pipeline will predict properties for 6,646,238 sources in HETDEX.


---

### Load models

In [15]:
if load_models_flag:
    if use_zeroth_model:
        star_clf        = pyc.load_model(gv.models_path + gv.star_model)  # star/no-star model
        cal_star_clf    = load(gv.models_path + gv.cal_str_model)  # calibrated model
    AGN_gal_clf         = pyc.load_model(gv.models_path + gv.AGN_gal_model)  #
    cal_AGN_gal_clf     = load(gv.models_path + gv.cal_AGN_gal_model)  # calibrated model
    radio_det_clf       = pyc.load_model(gv.models_path + gv.radio_model)  # without predicted AGN
    cal_radio_det_clf   = load(gv.models_path + gv.cal_radio_model)  # calibrated model
    redshift_reg        = pyr.load_model(gv.models_path + gv.full_z_model)  # to use on full sample
    if use_second_z_model:
        redshift_reg_2  = pyr.load_model(gv.models_path + gv.high_z_model)  # sources with predicted z >= 3.6

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [16]:
full_catalog_df = pyc.predict_model(AGN_gal_clf, data=full_catalog_df, probability_threshold=gv.AGN_thresh, raw_score=True, round=10)
full_catalog_df = full_catalog_df.drop(columns=['Score_0'])
full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_class', 'Score_1': 'Score_AGN'})
full_catalog_df['Score_AGN'] = np.around(full_catalog_df.loc[:, 'Score_AGN'], decimals=7)
pred_probs_AGN  = cal_AGN_gal_clf.predict(full_catalog_df.loc[:, 'Score_AGN'])
full_catalog_df['Prob_AGN'] = pred_probs_AGN

In [17]:
full_catalog_df = pyc.predict_model(radio_det_clf, data=full_catalog_df, probability_threshold=gv.radio_thresh, raw_score=True, round=10)
full_catalog_df = full_catalog_df.drop(columns=['Score_0'])
full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_radio', 'Score_1': 'Score_radio'})
full_catalog_df['Score_radio'] = np.around(full_catalog_df.loc[:, 'Score_radio'], decimals=7)
pred_probs_rad  = cal_radio_det_clf.predict(full_catalog_df.loc[:, 'Score_radio'])
full_catalog_df['Prob_radio'] = pred_probs_rad

In [18]:
full_catalog_df = pyr.predict_model(redshift_reg, data=full_catalog_df, round=8)
full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_Z'})
full_catalog_df['pred_Z'] = np.around(full_catalog_df.loc[:, 'pred_Z'], decimals=3)

In [19]:
redshift_tol    = 0.0
if use_second_z_model:
    full_catalog_df = pyr.predict_model(redshift_reg_2, data=full_catalog_df, round=6)
    filter_pred_z   = full_catalog_df.loc[:, 'pred_Z'] >= (gv.high_z_limit + redshift_tol)
    full_catalog_df.loc[:, 'pred_Z'] = full_catalog_df.loc[:, 'pred_Z'].mask(filter_pred_z, full_catalog_df.loc[filter_pred_z, 'Label'])
    full_catalog_df = full_catalog_df.drop(columns=['Label'])
    full_catalog_df.loc[:, 'pred_Z'] = np.around(full_catalog_df.loc[:, 'pred_Z'], decimals=3)

In [20]:
full_catalog_df['pred_prob_class']    = (full_catalog_df.loc[:, 'Prob_AGN']   >= gv.cal_AGN_thresh).astype(int)
full_catalog_df['pred_prob_radio']    = (full_catalog_df.loc[:, 'Prob_radio'] >= gv.cal_radio_thresh).astype(int)

In [21]:
full_catalog_df['pred_radio_AGN']     = (np.array(full_catalog_df.loc[:, 'pred_class'] == 1) & np.array(full_catalog_df.loc[:, 'pred_radio'] == 1)).astype(int)
full_catalog_df['Score_rAGN']         = full_catalog_df.loc[:, 'Score_AGN'] * full_catalog_df.loc[:, 'Score_radio']
full_catalog_df['pred_prob_rAGN']     = (np.array(full_catalog_df.loc[:, 'pred_prob_class'] == 1) & np.array(full_catalog_df.loc[:, 'pred_prob_radio'] == 1)).astype(int)
full_catalog_df['Prob_rAGN']          = full_catalog_df.loc[:, 'Prob_AGN'] * full_catalog_df.loc[:, 'Prob_radio']

rad_score_scaler                      = MinMaxScaler()
full_catalog_df['scaled_score_radio'] = rad_score_scaler.fit_transform(full_catalog_df.loc[:, 'Score_radio'].values.reshape(-1, 1))
full_catalog_df['scaled_score_rAGN']  = full_catalog_df.loc[:, 'Score_AGN'] * full_catalog_df.loc[:, 'scaled_score_radio']

In [22]:
full_catalog_df.loc[:, ['class', 'pred_class', 'LOFAR_detect', 'pred_radio', 'Z', 'pred_Z']].describe()

,class,pred_class,LOFAR_detect,pred_radio,Z,pred_Z
count,0.0,6.646238e+06,6.646238e+06,6.646238e+06,9662.000000,6.646238e+06
mean,NaN,1.181419e-01,4.742322e-02,6.701475e-02,0.400833,2.224591e+00
std,NaN,3.227761e-01,2.125424e-01,2.500476e-01,0.925536,7.251231e-01
min,NaN,0.000000e+00,0.000000e+00,0.000000e+00,-0.004140,1.600000e-02
25%,NaN,0.000000e+00,0.000000e+00,0.000000e+00,-0.000220,2.124000e+00
50%,NaN,0.000000e+00,0.000000e+00,0.000000e+00,-0.000010,2.262000e+00
75%,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.377580,2.262000e+00
max,NaN,1.000000e+00,1.000000e+00,1.000000e+00,7.028330,4.739000e+00


Obtain intermediate metrics

In [23]:
filter_radio_AGN_t      = np.array(full_catalog_df.loc[:, 'class'] == 1) & np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1)
filter_gal_AGN_t        = np.array(full_catalog_df.loc[:, 'class'] == 0) | np.array(full_catalog_df.loc[:, 'class'] == 1)
total_size              = len(full_catalog_df)
filter_AGN_t            = np.array(full_catalog_df.loc[:, 'class'] == 1)
num_AGN_t               = np.sum(filter_AGN_t)
num_gal_t               = np.sum(np.array(full_catalog_df.loc[:, 'class'] == 0))
num_radio_t             = np.sum(np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1))
num_radio_AGN_t         = np.sum(filter_radio_AGN_t)

In [24]:
# filter_radio_AGN_p      = np.array(full_catalog_df.loc[:, 'Score_AGN']          >= threshold_AGN) &\
#                             np.array(full_catalog_df.loc[:, 'Score_radio']      >= threshold_radio)
filter_radio_AGN_p      = np.array(full_catalog_df.loc[:, 'pred_class']      == 1) &\
                            np.array(full_catalog_df.loc[:, 'pred_radio']    == 1)
filt_hiz_rAGN_p         = filter_radio_AGN_p * np.array(full_catalog_df.loc[:, 'pred_Z'] >= gv.high_z_limit)
filter_AGN_p            = np.array(full_catalog_df.loc[:, 'pred_class']      == 1)
filter_radio_p          = np.array(full_catalog_df.loc[:, 'pred_radio']      == 1)
num_AGN_p               = np.sum(filter_AGN_p)
num_gal_p               = np.sum(np.array(full_catalog_df.loc[:, 'pred_class'] == 0))
num_radio_p             = np.sum(filter_radio_p)
num_radio_AGN_p         = np.sum(filter_radio_AGN_p)

### Select sources predicted to be Radio AGN

In [25]:
full_catalog_df         = full_catalog_df.loc[filter_radio_AGN_p]

In [26]:
full_catalog_df.loc[:, ['class', 'pred_class', 'LOFAR_detect', 'pred_radio', 'Z', 'pred_Z']].describe()

,class,pred_class,LOFAR_detect,pred_radio,Z,pred_Z
count,0.0,74704.0,74704.000000,74704.0,701.000000,74704.000000
mean,NaN,1.0,0.280132,1.0,0.670575,1.803116
std,NaN,0.0,0.449067,0.0,0.984706,0.860669
min,NaN,1.0,0.000000,1.0,-0.001320,0.016000
25%,NaN,1.0,0.000000,1.0,0.100000,0.997000
50%,NaN,1.0,0.000000,1.0,0.300000,2.050000
75%,NaN,1.0,1.000000,1.0,0.832930,2.425000
max,NaN,1.0,1.000000,1.0,6.879030,4.739000


Add individual metrics for redshift

In [27]:
full_catalog_df['Delta_z_N'] = np.around((full_catalog_df.loc[:, 'pred_Z'] - full_catalog_df.loc[:, 'Z']) /\
                            (1 + full_catalog_df.loc[:, 'Z']), decimals=3)

full_catalog_df['sigma_NMAD'] = np.around(1.48 * np.abs(full_catalog_df.loc[:, 'pred_Z'] - full_catalog_df.loc[:, 'Z']) /\
                            (1 + full_catalog_df.loc[:, 'Z']), decimals=3)

Numerical summary.

In [28]:
str_0_t   = f'Out of {total_size:,} initial sources in {used_area},\n'
str_1_t   = f'{num_gal_t:,} are confirmed to be galaxies. On the other side,\n'
str_2_t   = f'{num_AGN_t:,} are confirmed to be AGN. And, from the AGN,\n'
str_3_t   = f'{num_radio_AGN_t:,} are detected in radio.'

str_0_p   = f'Out of {num_radio_AGN_t:,} initial radio-detected AGN in {used_area},\n'
str_1_p   = f'{num_gal_p:,} are predicted to be galaxies. On the other side,\n'
str_2_p   = f'{num_AGN_p:,} are predicted to be AGN. And, from the predicted AGN,\n'
str_3_p   = f'{num_radio_AGN_p:,} are predicted to be detected in radio.'

In [29]:
print('-' * 60)
print(str_0_t + str_1_t + str_2_t + str_3_t)
print('-' * 60)
print(str_0_p + str_1_p + str_2_p + str_3_p)
print('-' * 60)

------------------------------------------------------------
Out of 6,646,238 initial sources in HETDEX,
0 are confirmed to be galaxies. On the other side,
0 are confirmed to be AGN. And, from the AGN,
0 are detected in radio.
------------------------------------------------------------
Out of 0 initial radio-detected AGN in HETDEX,
5,861,039 are predicted to be galaxies. On the other side,
785,199 are predicted to be AGN. And, from the predicted AGN,
74,704 are predicted to be detected in radio.
------------------------------------------------------------


In [30]:
cols_4_table = show_columns = ['Name', 'RA_ICRS', 'DE_ICRS', 'TYPE', 'band_num', 'class', 'pred_class',\
                               'pred_prob_class', 'Score_AGN', 'Prob_AGN', 'LOFAR_detect', 'pred_radio',\
                               'pred_prob_radio', 'Score_radio', 'Prob_radio', 'radio_AGN', 'pred_radio_AGN',\
                               'pred_prob_rAGN', 'Score_rAGN', 'Prob_rAGN', 'Z', 'pred_Z']

In [31]:
cols_4_export_S82    = ['Total_flux_VLASS', 'Fint_VLAS82', 'Stotal_TGSS', 'FEP']
cols_4_export_HETDEX = ['Total_flux_VLASS', 'Sint_LOFAR', 'Stotal_TGSS', 'FEP', 'TotalFlux_LoLSS']
cols_4_export_COSMOS = ['Total_flux_VLASS', 'Stotal_TGSS', 'FEP', 'Flux_COSMOSVLA3']

cols_4_exp_all       = {'S82': cols_4_export_S82, 'HETDEX': cols_4_export_HETDEX, 'COSMOS': cols_4_export_COSMOS}

cols_photo           = ['W1mproPM', 'W2mproPM', 'gmag', 'rmag', 'imag', 'zmag', 'ymag',\
                        'Jmag', 'Hmag', 'Kmag', 'W3mag', 'W4mag']

cols_4_export        = cols_4_table + cols_4_exp_all[used_area] + cols_photo

In [32]:
filter_new_rAGN = full_catalog_df.loc[:, 'radio_AGN'] == 0
if predict_only_hi_z:
    filter_high_z   = full_catalog_df.loc[:, 'pred_Z']    >= gv.high_z_limit
elif not predict_only_hi_z:
    filter_high_z   = np.ones_like(full_catalog_df.loc[:, 'pred_Z']).astype(bool)

In [33]:
full_catalog_df.loc[:, 'TYPE'] = full_catalog_df.loc[:, 'TYPE'].mask(full_catalog_df.loc[:, 'TYPE'] == '    ', 'CCCC')

In [34]:
display(full_catalog_df.loc[filter_high_z, cols_4_export].sort_values(by=['pred_Z'], ascending=False).head(15))

,Name,RA_ICRS,DE_ICRS,TYPE,band_num,class,pred_class,pred_prob_class,Score_AGN,Prob_AGN,LOFAR_detect,pred_radio,pred_prob_radio,Score_radio,Prob_radio,radio_AGN,pred_radio_AGN,pred_prob_rAGN,Score_rAGN,Prob_rAGN,Z,pred_Z,Total_flux_VLASS,Sint_LOFAR,Stotal_TGSS,FEP,TotalFlux_LoLSS,W1mproPM,W2mproPM,gmag,rmag,imag,zmag,ymag,Jmag,Hmag,Kmag,W3mag,W4mag
2217066,J132959.63+470313.5,202.498444,47.053791,CCCC,8,NaN,1,1,0.639387,0.653307,0,1,1,0.499585,0.495449,0,1,1,0.319428,0.323681,NaN,4.739,15.21,17.52,11.18,33.276237,12.91,19.080999,19.478001,23.299999,21.238400,18.909599,18.847300,18.512400,17.450001,17.24,16.59,16.67,14.62
4786343,J151237.08+543229.7,228.154526,54.541595,CCCC,8,NaN,1,1,0.991423,0.987216,0,1,1,0.497309,0.288662,0,1,1,0.493043,0.284972,NaN,4.739,15.21,17.52,11.18,57.900002,12.91,19.594999,19.809999,23.299999,21.349899,19.876600,19.917700,19.839500,17.450001,17.24,16.59,16.67,14.62
5389236,J132135.74+560140.7,200.398926,56.027981,CCCC,8,NaN,1,1,0.700025,0.708180,0,1,1,0.499502,0.487396,0,1,1,0.349664,0.345164,NaN,4.670,15.21,17.52,11.18,57.900002,12.91,19.286999,19.586000,23.299999,20.695801,19.471300,19.520700,19.297001,17.450001,17.24,16.59,16.67,14.62
4718130,J104718.00+523820.3,161.825012,52.638958,CCCC,8,NaN,1,1,0.969608,0.961524,0,1,1,0.501565,0.679283,0,1,1,0.486321,0.653148,NaN,4.662,15.21,17.52,11.18,57.900002,12.91,18.886999,19.282000,23.299999,21.132999,19.729700,19.122999,18.879900,17.450001,17.24,16.59,16.67,14.62
580596,J110540.27+491458.0,166.417801,49.249458,CCCC,6,NaN,1,1,0.959746,0.950851,0,1,1,0.500324,0.566810,0,1,1,0.480184,0.538952,NaN,4.641,15.21,17.52,11.18,57.900002,12.91,19.177000,19.152000,23.299999,20.875299,19.575500,18.985399,18.729601,17.450001,17.24,16.59,16.67,14.62
2131747,J134907.87+490230.6,207.282776,49.041828,CCCC,8,NaN,1,1,0.993965,0.990590,0,1,1,0.499585,0.495449,0,1,1,0.496570,0.490787,NaN,4.607,15.21,17.52,11.18,57.900002,12.91,19.248999,19.316000,23.299999,21.172100,19.945101,19.344000,19.101601,17.450001,17.24,16.59,16.67,14.62
5210113,J111754.83+511102.8,169.478455,51.184124,CCCC,8,NaN,1,1,0.994896,0.991870,0,1,1,0.501565,0.679283,0,1,1,0.499005,0.673761,NaN,4.605,15.21,17.52,11.18,57.900002,12.91,18.915998,19.189001,23.299999,21.139799,19.712999,19.057301,18.818300,17.450001,17.24,16.59,16.67,14.62
4125590,J121833.84+533549.4,184.641006,53.597076,CCCC,8,NaN,1,1,0.872739,0.865296,0,1,1,0.501565,0.679283,0,1,1,0.437735,0.587781,NaN,4.605,15.21,17.52,11.18,57.900002,12.91,18.942999,19.278999,23.299999,21.390900,20.093700,19.422501,19.132601,17.450001,17.24,16.59,16.67,14.62
3410222,J132514.21+534447.1,201.309235,53.746429,CCCC,8,NaN,1,1,0.941747,0.932154,0,1,1,0.501565,0.679283,0,1,1,0.472347,0.633197,NaN,4.599,15.21,17.52,11.18,57.900002,12.91,18.823999,19.125999,23.299999,20.935801,19.635599,19.379000,19.478600,17.450001,17.24,16.59,16.67,14.62
2418103,J110242.02+533950.5,165.675064,53.664040,CCCC,8,NaN,1,1,0.447773,0.474833,0,1,1,0.499585,0.495449,0,1,1,0.223701,0.235256,NaN,4.594,15.21,17.52,11.18,57.900002,12.91,19.260000,19.183001,23.299999,21.444000,20.122101,19.566299,19.235500,17.450001,17.24,16.59,16.67,14.62


In [35]:
if save_preds_flag:
    full_catalog_df.loc[filter_high_z, cols_4_export].sort_values(by=['pred_Z'],
     ascending=False).to_csv(gv.preds_path + f'predicted_rAGN_{used_area}.csv')

---